In [1]:
import pandas as pd
import distutils
from datetime import datetime

# Backup

In [2]:
# Currently done manually find way to back up loader folder and data folder 
path = "AutoBack\ " + datetime.now().strftime("%d-%m-%y_%H.%M.%S")
distutils.dir_util.copy_tree('Collection',path + '/Collection')
distutils.dir_util.copy_tree('Loader',path + '/Loader')

['AutoBack\\ 03-03-24_12.12.31/Loader\\CardsToBeAdded.csv',
 'AutoBack\\ 03-03-24_12.12.31/Loader\\CardsToBeRemoved.csv']

# Load Data

## All Card DataBase

In [3]:
CardData = pd.read_csv('CleanData/Cards.csv',index_col = 'uuid')

C:\Users\helal\AppData\Local\Temp\ipykernel_3784\4144974011.py:1: DtypeWarning: Columns (4,8,10,13,17,21,24,48,54,62,67,69) have mixed types. Specify dtype option on import or set low_memory=False.
  CardData = pd.read_csv('CleanData/Cards.csv',index_col = 'uuid')


## Collection

In [4]:
Collection = pd.read_csv('Collection/Raw/Collection.csv',index_col = 'uuidF')
ValueBinder = pd.read_csv('Collection/Raw/ValueBinder.csv',index_col = 'uuidF')
EDHBinder = pd.read_csv('Collection/Raw/EDHBinder.csv',index_col = 'uuidF')
BulkBinder = pd.read_csv('Collection/Raw/BulkBinder.csv',index_col = 'uuidF')

## Cards Added

In [5]:
CardsAdded = pd.read_csv('Loader/CardsToBeAdded.csv')

CardsAdded['Foil'].fillna(0,inplace = True)
# should probably just find non empty cells instead of specifically finding things called "foil"
CardsAdded.loc[(CardsAdded['Foil'] == 'FOIL'),'Foil'] = 1
CardsAdded['Foil'] = CardsAdded['Foil'].astype(int) 
CardsAdded['CollectionID'] = CardsAdded['Card'] +  CardsAdded['Set ID']

## Cards Removed

In [6]:
CardsRemoved = pd.read_csv('Loader/CardsToBeRemoved.csv')

CardsRemoved['Foil'].fillna(0,inplace = True)
CardsRemoved.loc[(CardsRemoved['Foil'] == 'FOIL'),'Foil'] = 1

CardsRemoved['Foil'] = CardsRemoved['Foil'].astype(int) 

CardsRemoved['CollectionID'] = CardsRemoved['Card'] +  CardsRemoved['Set ID']

#### Raise Exception when Removed Card not in Collection 

In [7]:
CardsAdded

,Card,Set ID,Quantity,Foil,CollectionID
0,Abhorrent Overlord,THS,1,0,Abhorrent OverlordTHS
1,Abominable Treefolk,MH1,1,0,Abominable TreefolkMH1
2,Abrade,2XM,1,0,Abrade2XM
3,Abrupt Decay,MM3,2,0,Abrupt DecayMM3
4,Abundance,DDR,1,0,AbundanceDDR
...,...,...,...,...,...
1819,Zealous Conscripts,MM3,1,0,Zealous ConscriptsMM3
1820,Zedruu the Greathearted,C16,1,0,Zedruu the GreatheartedC16
1821,"Zegana, Utopian Speaker",RNA,2,0,"Zegana, Utopian SpeakerRNA"
1822,Zurgo Bellstriker,DTK,1,0,Zurgo BellstrikerDTK


# Identify

In [8]:
Identifier = CardData['CollectionID'].copy()
Identifier = Identifier.reset_index()

# Raise Exception for when a collection ID is not in the Identifer list prompting user to update DATA 

## Unidentifiable 

In [9]:
acollectionID = CardsAdded['CollectionID'].tolist()
rcollectionID = CardsRemoved['CollectionID'].tolist()

#### Querry relevant cards

In [10]:
afilt = Identifier['CollectionID'].isin(acollectionID)
rfilt = Identifier['CollectionID'].isin(rcollectionID)

aquery = Identifier[afilt].copy()
rquery = Identifier[rfilt].copy()

#### Locate Unidentifiable Cards

In [11]:
afilt = aquery['CollectionID'].duplicated()
rfilt = rquery['CollectionID'].duplicated()

aunidentifiable = aquery[afilt]['CollectionID'].tolist()
runidentifiable = rquery[rfilt]['CollectionID'].tolist()

#### Remove and Save Unidentifiable Cards for Data

In [12]:
afilt = CardsAdded['CollectionID'].isin(aunidentifiable)
rfilt = CardsRemoved['CollectionID'].isin(runidentifiable)

UnidentifiableAdded = CardsAdded[afilt].copy()
UnidentifiableRemoved = CardsRemoved[rfilt].copy()

IdentifiableAdded = CardsAdded[~afilt].copy()
IdentifiableRemoved = CardsRemoved[~rfilt].copy()

### Handling Unidentifiable Cards

In [13]:
## Pause Code
## Ask to insert Card Collector Number 
## Resume Code

## Assign uuid

In [14]:
# Added
IdedCardsAdded = IdentifiableAdded.merge(Identifier, how= 'left', on = 'CollectionID', copy = True)

IdedCardsAdded['uuidF'] = IdedCardsAdded['uuid'] + IdedCardsAdded['Foil'].astype('str')
IdedCardsAdded.set_index('uuidF',inplace = True)

IdedCardsAdded.drop(['CollectionID','Card','Set ID','Foil','uuid'],axis = 1,inplace = True)


# Removed
IdedCardsRemoved = IdentifiableRemoved.merge(Identifier, how= 'left', on = 'CollectionID', copy = True)

IdedCardsRemoved['uuidF'] = IdedCardsRemoved['uuid'] + IdedCardsRemoved['Foil'].astype('str')
IdedCardsRemoved.set_index('uuidF',inplace = True)

IdedCardsRemoved.drop(['CollectionID','Card','Set ID','Foil','uuid'],axis = 1,inplace = True)

# Assign Binder

In [15]:
def assign_binder(uuidf):
    global CardData
    uuid =  uuidf[0:len(uuidf)-1]
    foiling = int(uuidf[-1])
    
    card = CardData.loc[uuid].copy()
    card['price'] = card['normal price'] + (card['foil price'] - card['normal price'])*foiling

    global Collection 
    if uuidf in Collection.index:
        return 'Collection'
    
    # Special 
    
    # Land
    if card['type'] == ('Land'):
        return 'Land'
    
    # Value
    if card['price'] > 5:
        return 'Value'

    # EDH
    if card['edhrecRank'] < 1500:
        return 'EDH'
    # Bulk
    if card['rarity'] in ['rare','mythic']:
        return 'Bulk'
    # Other
    
    return 'not Sorted'
    

    

## Assign Cards

In [16]:
BeingAssigned = IdedCardsAdded.copy()
BeingAssigned['Binder'] = BeingAssigned.index.map(assign_binder)

filt = BeingAssigned['Binder'] == 'Collection'

AssignedCards = BeingAssigned[~filt].copy()
CardsInCollection = BeingAssigned[filt].copy()

## insert into collection 

In [17]:
mask = Collection.index.isin(CardsInCollection.index)

# Increase QTY for Existing Cards
Collection['Quantity'] = Collection['Quantity'].add(CardsInCollection['Quantity'], fill_value=0)
# Append New Cards to Collection
Collection = pd.concat([Collection.astype(AssignedCards.dtypes), AssignedCards.astype(Collection.dtypes)])

# Insert in Binder

## Defintions

In [34]:
def add_page(binder, location):
    mask = binder['Page'] > location
    binder.loc[mask,'Page'] = binder.loc[mask,'Page'] + 1   

# Page is added sepratly not sure if i have this return page + 1 would still manipulate the page, heck im actually not returning anything 
# and im already manipulating the inputed binder 

In [19]:
def add_card(binder, card, uuid,page, slot):
    global CardData
    
    Page = page
    Slot = slot
    Scolor = CardData.loc[uuid,'order']
    SmanaValue = CardData.loc[uuid,'manaValue']
    
    l = len(binder)
    binder.loc[l] = [Page,Slot,Scolor,SmanaValue]
    binder.rename(index = {l: card.name},inplace = True)    

In [20]:

def add(binder, cards):
    binder = binder.copy()
    global CardData

    for i, card in cards.iterrows():

        uuid = i[0:len(i)-1]
        card_data = CardData.loc[uuid]
        color_section = card_data['order']
        mv_section = card_data['manaValue']


        # if section exists 
        filt = (binder['Scolor'] == color_section) & (binder['SmanaValue'] == mv_section)
        slice = binder[filt]

        if len(slice) > 0:
            page = slice['Page'].max() 
            slot = slice.loc[(slice['Page'] == page),'Slot'].max()

            if slot < 9:
                slot = slot + 1
            else:
                add_page(binder,page)
                page = page + 1
                slot = 1
                
            add_card(binder, card, uuid,page, slot)
            continue 

        # if section doesnt exist
        filt = (binder['Scolor'] <= color_section)
        color_slice = binder[filt]    
        closet_color = color_slice['Scolor'].max()

        if not pd.isna(closet_color):
            
            # put this in an if statemnt the checks if closest color is color
            filt = (binder['Scolor'] == closet_color) 
            color_slice = binder[filt]

            filt = (color_slice['SmanaValue'] < mv_section)
            slice = color_slice[filt]

            if closet_color != color_section:
                page = color_slice['Page'].max()
            #Color Section Exist no CMC > Cards
            elif slice.empty:
                page = color_slice['Page'].min() - 1
            #Color Sections Exists CMC > Card also exist
            else: 
                page = slice['Page'].max()
        else: 
            page = 0 
        
        add_page(binder,page)
        page = page + 1 
        slot = 1
        add_card(binder, card, uuid,page, slot)

    return binder.sort_values(['Page','Slot'])    

In [21]:
def remove(Binder, Cards):
    pass

## inserting into binder

### Land

In [22]:
LandList = AssignedCards[AssignedCards['Binder'] == 'Land']

### Value

In [23]:
filt = AssignedCards['Binder'] == 'Value'
cards = AssignedCards[filt]
ValueBinder = add(ValueBinder, cards)

### EDH

In [24]:
filt = AssignedCards['Binder'] == 'EDH'
cards = AssignedCards[filt]
EDHBinder = add(EDHBinder, cards)

### Bulk

In [25]:
filt = AssignedCards['Binder'] == 'Bulk'
cards = AssignedCards[filt]
BulkBinder = add(BulkBinder, cards)

## Other

In [26]:
otherlist = AssignedCards[AssignedCards['Binder'] == 'not Sorted']

# Remove Cards

In [ ]:
# Subtract Cards Removed From Collection 
# Check if Qty = 0 
# if Q = 0 remove cards from binder 
#     # to do this append all binders together and merge with  filter by for QTY = 0 after by ussing the uuid in collection

In [32]:
IdedCardsRemoved

,Quantity
uuidF,


In [33]:
Collection

,Quantity,Binder
uuidF,,
56c53552-6f92-51fa-974a-40edcb6ca76e0,1,Bulk
e72320c5-aa3a-5fd9-88d0-56f0d71f9e7b0,1,not Sorted
01df4e32-7fe4-5b8d-9460-0f08fa29153e0,1,EDH
b11a2ad4-06b7-508a-925e-37507ba371c40,2,EDH
5f0ba7fd-1290-5668-b953-4d0876f88f270,1,EDH
...,...,...
2854d82c-4a8b-523b-bbaf-7102d93fc5cd0,1,EDH
2349cff1-4218-55c6-bff0-819bc15510480,1,Bulk
ccdabe5f-edcb-561c-b2c8-95ff1bba5c110,2,Bulk


# Export

In [27]:
#Export to collection Folder
Collection.to_csv('Collection/Raw/Collection.csv')
ValueBinder.to_csv('Collection/Raw/ValueBinder.csv')
EDHBinder.to_csv('Collection/Raw/EDHBinder.csv')
BulkBinder.to_csv('Collection/Raw/BulkBinder.csv')

LandList.to_csv('Collection/Raw/not_Sorted/Land.csv')
otherlist.to_csv('Collection/Raw/not_Sorted/other.csv')

# unidtifiable 


In [28]:
# Reset Loader
CardsAdded.drop(CardsAdded.index, inplace = True)
CardsAdded.drop('CollectionID',axis = 1, inplace = True)
CardsAdded.to_csv('Loader/CardsToBeAdded.csv')

CardsRemoved.drop(CardsRemoved.index, inplace = True)
CardsRemoved.drop('CollectionID',axis = 1, inplace = True)
CardsRemoved.to_csv('Loader/CardsToBeRemoved.csv')

In [29]:
#Return Unidentified List 
UnidentifiableAdded.drop('CollectionID',axis = 1, inplace = True)
UnidentifiableAdded.to_csv('Loader/Unidentified/UnidentifiableAdded.csv')

UnidentifiableRemoved.drop('CollectionID',axis = 1, inplace = True)
UnidentifiableRemoved.to_csv('Loader/Unidentified/UnidentifiableRemoved.csv')

In [30]:
CardData.loc['7d40e039-e287-53e8-8b55-08a425c62aab','name']

'Blood Speaker'

In [31]:
EDHBinder

,Page,Slot,Scolor,SmanaValue
uuidF,,,,
bd1d5176-824c-56d7-a682-770b2ba91b9f0,1.0,1.0,1.0,1.0
b15d098d-2855-53fc-ab42-9047960556141,1.0,2.0,1.0,1.0
488fa58a-db5b-520e-84ae-0c79a2988f480,1.0,3.0,1.0,1.0
ed8a0c79-e338-5d0b-8a26-7a4ccb639bfe0,1.0,4.0,1.0,1.0
5fc178d7-de62-525c-a83d-cc9090d76f150,2.0,1.0,1.0,2.0
...,...,...,...,...
7b374d85-9ca5-5346-9b9c-bf49de99706d0,73.0,4.0,28.0,6.0
f429b606-9aa0-5c99-bc6a-4eeab4fabc120,74.0,1.0,28.0,7.0
3d769d2e-73ed-5ba9-9bd5-545f82c81f120,74.0,2.0,28.0,7.0
